In [1]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import findspark
import os

os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'

findspark.init()
findspark.find()

'/usr/local/lib/python3.8/dist-packages/pyspark'

In [29]:
spark = SparkSession.builder \
                    .master("local") \
                    .appName("Learning DataFrames") \
                    .getOrCreate()
events = spark.read.json("/user/master/data/events/date=2022-05-25")
# events.show(10)

---

In [30]:
events.printSchema()

root
 |-- event: struct (nullable = true)
 |    |-- admins: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- channel_id: long (nullable = true)
 |    |-- datetime: string (nullable = true)
 |    |-- media: struct (nullable = true)
 |    |    |-- media_type: string (nullable = true)
 |    |    |-- src: string (nullable = true)
 |    |-- message: string (nullable = true)
 |    |-- message_channel_to: long (nullable = true)
 |    |-- message_from: long (nullable = true)
 |    |-- message_group: long (nullable = true)
 |    |-- message_id: long (nullable = true)
 |    |-- message_to: long (nullable = true)
 |    |-- message_ts: string (nullable = true)
 |    |-- reaction_from: string (nullable = true)
 |    |-- reaction_type: string (nullable = true)
 |    |-- subscription_channel: long (nullable = true)
 |    |-- tags: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- user: string (nullable = true)
 |-- event_type: s

In [34]:
events.select('event_type').distinct().show()

+------------+
|  event_type|
+------------+
|    reaction|
|subscription|
|     message|
+------------+



In [41]:
events \
    .filter(F.col('event_type') == 'reaction') \
    .groupby('event.reaction_from') \
    .count() \
    .select(F.max('count')) \
    .show()

+----------+
|max(count)|
+----------+
|        11|
+----------+



In [24]:
events \
    .select(
        F.col('*'),
        F.hour('event.datetime'),
        F.minute('event.datetime'),
        F.second('event.datetime'),
    ) \
    .sort(F.col('event.datetime')) \
    .show(10)

+--------------------+------------+--------------------+----------------------+----------------------+
|               event|  event_type|hour(event.datetime)|minute(event.datetime)|second(event.datetime)|
+--------------------+------------+--------------------+----------------------+----------------------+
|{[19342], 987160,...|     message|                  23|                    57|                    38|
|{null, null, 2022...|subscription|                  23|                    56|                    58|
|{[26358], 247511,...|     message|                  23|                    56|                    53|
|{[79792], 748847,...|     message|                  23|                    56|                    27|
|{null, null, 2022...|subscription|                  23|                    53|                    42|
|{null, null, 2022...|subscription|                  23|                    53|                    14|
|{[151897], 396845...|     message|                  23|                 

In [28]:
events \
    .filter(F.col('event.message_to').isNotNull()) \
    .count()

221

In [48]:
events \
    .groupby('event_type') \
    .count() \
    .show()

+------------+------+
|  event_type| count|
+------------+------+
|    reaction|364815|
|subscription|700913|
|     message|   909|
+------------+------+



In [85]:
events.first()

Row(event=Row(admins=None, channel_id=None, datetime='2022-05-25 17:04:31', media=None, message=None, message_channel_to=None, message_from=None, message_group=None, message_id=None, message_to=None, message_ts=None, reaction_from=None, reaction_type=None, subscription_channel=913681, tags=None, user='48430'), event_type='subscription')

In [90]:
# events.rdd.mapValues(lambda r: r[:3]).first()
events.rdd \
    .keyBy(lambda row: row.event_type) \
    .groupByKey() \
    .reduce()


('subscription',
 Row(event=Row(admins=None, channel_id=None, datetime='2022-05-25 17:04:31', media=None, message=None, message_channel_to=None, message_from=None, message_group=None, message_id=None, message_to=None, message_ts=None, reaction_from=None, reaction_type=None, subscription_channel=913681, tags=None, user='48430'), event_type='subscription'))

In [51]:
events.rdd.reduceByKey(len).collect()

23/08/01 18:55:12 ERROR Executor: Exception in task 0.0 in stage 50.0 (TID 47)2]
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 830, in main
    process()
  File "/usr/local/lib/python3.8/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 820, in process
    out_iter = func(split_index, iterator)
  File "/usr/local/lib/python3.8/dist-packages/pyspark/rdd.py", line 5405, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/lib/python3.8/dist-packages/pyspark/rdd.py", line 828, in func
    return f(iterator)
  File "/usr/local/lib/python3.8/dist-packages/pyspark/rdd.py", line 3964, in combineLocally
    merger.mergeValues(iterator)
  File "/usr/local/lib/python3.8/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py", line 258, in mergeValues
    d[k] = comb(d[k], v) if k in d else creator(v

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 50.0 failed 1 times, most recent failure: Lost task 0.0 in stage 50.0 (TID 47) (172.16.0.18 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 830, in main
    process()
  File "/usr/local/lib/python3.8/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 820, in process
    out_iter = func(split_index, iterator)
  File "/usr/local/lib/python3.8/dist-packages/pyspark/rdd.py", line 5405, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/lib/python3.8/dist-packages/pyspark/rdd.py", line 828, in func
    return f(iterator)
  File "/usr/local/lib/python3.8/dist-packages/pyspark/rdd.py", line 3964, in com

In [ ]:
events.

In [6]:
spark.stop()